# Twitter Scrap

In [ ]:
!pip install snscrape

import pandas as pd
import snscrape.modules.twitter as sntwitter
import numpy as np
import matplotlib. pyplot as plt
import seaborn as sns
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import 'WordNetLemmatizer
frun nltk.stem.porter import PorterStemmer

import string
import re
import textblob
from textblob import TextBlob

from wordcloud import WordCloud, STOPWORDS

from wordcloud import ImageColorGenerator

import warnings
%matplotlib inline

import os

#using OS library to call CLI commands in python
os.system("snscrape --json1 --max-results 10000 --since 2023-03-13 twitter-search 'chatGPT4' > text-chatGPT4-tweets.json")

import pandas as pd

#create a pandas database
tweets_df_chatGPT4 = pd.read_json('text-chatGPT4-tweets.json', lines=True)

tweets_df_chatGPT4.head()

#Data loading
df_chatGPT4 = tweets_df_chatGPT4[['date', 'rawContent'.'renderedContent', 'user', 'replyCount','retweetCount','likeCount','lang','place','hashtags','viewCount']]
print(df_chatGPT4.shape)

# Twitter data cleaning, preprocessing and eda
df2 = df_chatGPT4.drop_duplicates('renderedContent')
print(df2.shape)

df2.head()

df2.info()

df2.date.value_counts()

# if want to convert to excel then
# df2.to_excel('twitter_data.xlsx', index=False)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#Heat map for missing values
plt.figure(figsize=(17,5))
sns.heatmap(df2.isnull(), char=True, yticklabels=False)
plt.xlabel("Column_Name", size=14, weight="bold")
plt.title("Place of missing values in column", fontweight="bold", size=17)


1st is theory

2nd exp - Data Cleaning and Preprocessing Techniques for Enhanced Analysis and
Modeling

In [ ]:
import nltk
nltk.download('stopwords')

import re, numpy as np, pandas as pd
from pprint import pprint

import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
stop_words =stopwords.words('english')

import spacy
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

df =pd.read_csv('google.csv')

df.head(5)

df=df[['Text']]

df

def sent_to_words(sentences):
  for sent in sentences:
    sent=re.sub('\s+',' ',sent)
    sent=re.sub("\'","",sent)
    sent=gensim.utils.simple_preprocess(str(sent), deacc=True)
    yield(sent)

All_reviews=df.Text.values.tolist()
review_words=list(sent_to_words(All_reviews))
print(review_words[:1])

bigram=gensim.models.Phrases(review_words, min_count=5, threshold=10)
trigram=gensim.models.Phrases(bigram[review_words], threshold=10)
bigram_mod=gensim.models.phrases.Phraser(bigram)
trigram_mod=gensim.models.phrases.Phraser(trigram)

def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN','ADJ','VERB','ADV']):
  texts==[[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
  texts=[bigram_mod[doc] for doc in texts]
  texts=[trigram_mod[bigram_mod[doc]] for doc in texts]
  texts_out=[]
  nlp=spacy.load('en_core_web_sm', disable=['parser','ner'])
  for sent in texts:
    doc=nlp(" ".join(sent))
    texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
  texts_out= [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]
  return texts_out

data_final =process_words(review_words)

print(data_final)


# LDA Topic model
id2word =  corpora.Dictionary(data_final)

corpus =[id2word.doc2bow(text) for text in data_final]

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=7, random_state=100, update_every=1, chunksize=10, passes=10, alpha='symmetric', iterations =100, per_word_topics=True)

pprint(lda_model.print_topics())

#WORD CLOUDS OF TOP N KEYWORDS IN EACH TOPIC
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

cols=[color for name, color in mcolors.TABLEAU_COLORS.items()]

cloud=WordCloud(stopwords=stop_words, background_color='white', width=2500, height=1800, max_words=10, colormap='tab10', color_func=lambda *args, **kwargs: cols[i], prefer_horizontal=1.0)

topics=lda_model.show_topics(formatted=False)

fig, axes =plt.subplots(3,2, figsize=(10,10), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
  fig.add_subplot(ax)
  topic_words=dict(topics[i][1])
  cloud.generate_from_frequencies(topic_words, max_font_size=300)
  ax.imshow(cloud)
  ax.set_title('Topic '+str(i),  fontdict=dict(size=16))
  ax.axis("off")

plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show

word_dict={};
for i in range(7):
  words=lda_model.show_topic(i, topn=20)
  word_dict['Topic #' + '{:02d}'.format(i)] = [i[0] for i in words]
pd.DataFrame(word_dict)

from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook

topic_weights =[]
for i, row_list in enumerate(lda_model[corpus]):
  topic_weights.append([w for i, w in row_list[0]])

arr=pd.DataFrame(topic_weights).fillna(0).values

arr=arr[np.amax(arr, axis=1)>0.35]

topic_num =np.argmax(arr, axis=1)

tsne_model =TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
tsne_lda=tsne_model.fit_transform(arr)

output_notebook()
n_topics=7
mycolors=np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])
plot = figure(title="t-SNE Clustering of {} LDA Topics".format(n_topics))
plot.scatter(x=tsne_lda[:,0], y=tsne_lda[:,1], color=mycolors[topic_num])

show(plot)


3rd exp - Data cleaning and storage: preprocess, filter and store social
media data for business

In [ ]:
df = pd.read_csv("restaurants-dataset/swiggy.csv")

df.shape

df.columns

df.sample(5)

df.info()

df.head(3)

# data cleaning - missing value handling
#Checking number of null values in each column
df.isnull().sum()

#Checking the percentage of missing values in each column
round(100*(df.isnull().sum()/len(df.index)), 2)

# 0.6 or 0.9 are very minute, so we remove them
df.dropna(subset=['name','rating','rating_count','cost','cuisine','address'],how='any')

#Filling the null values in the lic_no with Mode based on each licence number
m=df['lic_no'].mode()
df.fillna(m)

#Filtering the null rows from null valued columns
df = df[-df["name"].isnull()]
df = df[-df["rating"].isnull()]
df = df[-df["rating_count"].isnull()]
df = df[-df["cost"].isnull()]
df = df[-df["cuisine"].isnull()]
df = df[-df["address"].isnull()]
df = df[-df["lic_no"].isnull()]

round(100*(df.isnull().sum()/len(df.index)), 2)

#Now the data is cleanes without any missing values
#Calculating the data loss
100-round(100*len(df.index)/148541,2)

#We lost 0.1% data in the data cleaning process

#check for duplicated column
df.duplicated().sum()

#3. Check for appropriate datatype

# # for rating column
# replace '--' with 0 and typecasted to float

def fun(column):
    rating_int = []
    for i in column:
        if i=='--':
            rating_int.append(0)
        else:
            rating_int.append(float(i))
    return rating_int

df['rating'] = fun(df['rating'])
df['rating'].replace(0.0,round(df['rating'].mean(),2), inplace=True)

df['rating_count'].value_counts()

# for rating counts
# convert Too Few Ratings to 1 and whole typecasted to float

def fun(n):
    try:
        num = n.split(" ")[0]
        if num.split('+')[0].isdigit():
            return float(num.split('+')[0])
        else:
            return 1
    except:
        return n

df['rating_count']=df['rating_count'].apply(fun)

# for cost column
# just remove the ₹ sign and convert to float

def fun(a):
    try:
        return float(a.split('₹')[1])
    except:
        return 0

df['cost'] = df['cost'].apply(fun)

#Data Cleaning is completed now




4th exp - Exploratory Data Analysis in Social Media Analytics: Unveiling Insights
from Swiggy Restaurant Data

In [ ]:
# on 3rd exp above cleaning data ..do that and then eda
# Exploratory data Analysis

# Mathematical summary
df.describe()

# 1. Restaurant with Maximum Rating in Abohar
df1 = df[df['city']=='Abohar']
max_rating_in_city = df1.groupby('name')['rating'].max().sort_values(ascending=False).head(5)
max_rating_in_city

plt.figure(figsize=(10, 7))
max_rating_in_city.plot.bar()
plt.ylabel("Rating in number")
plt.xlabel("Restaurent name")
plt.title("Restaurant with Maximum Rating in Abohar City of Punjab")
plt.show()


# 2. Number of cities (including subregions) where swiggy is having their restaurants listed?
len(df.groupby('city'))

# 3. Number of cities (don't include subregions) where swiggy is having their restaurants listed?
df['city'].apply(lambda x: x.split(",")[-1]).nunique()

# 4. The city with the maximum number of restaurants listed on Swiggy?
df.groupby('city').size().sort_values(ascending=False).reset_index().iloc[0,0]

# 5. Restaurant chain with maximum number of branches?
df['name'].value_counts().sort_values(ascending=False).index[0]

# 6. Top 10 cities as per the number of restaurants listed?
top_10_cities = df.groupby('city')['name'].count().sort_values(ascending=False).head(10)
top_10_cities

plt.figure(figsize=(15, 8))
top_10_cities.plot.bar()
plt.ylabel("number of restaurants")
plt.xlabel("Cities")
plt.title("Top 10 cities as per the number of restaurants")
plt.show()

# 7. Most Popular Cuisines served throughout the dataset?
a = []
for i in df['cuisine'].values:
    try:
        for j in i.split(','):
            a.append(j)

    except:
        a.append(i)

pd.DataFrame({"all_cuisine":a}).value_counts().head(1).index[0]

# 8. Which city is having the least expensive restaurant in terms of cost?
df.groupby('city')['cost'].mean().sort_values().index[0]

# 9. Top 5 most popular restaurant chains in India?
Top_5_most_popular_restaurant_chains = df.groupby('name')['rating'].mean().sort_values(ascending=False).head(5)
Top_5_most_popular_restaurant_chains

plt.figure(figsize=(15, 8))
Top_5_most_popular_restaurant_chains.plot.bar()
plt.ylabel("rating")
plt.xlabel("Restaurent")
plt.title("Top 5 most popular restaurant chains in India")
plt.show()



df2 = df[df['city'].str.contains('Bangalore')]

# 10. Which restaurant in Banglore has the most number of people visited(take a number of reviews for reference)?

df2 = df[df['city'].str.contains('Bangalore')]
df2.groupby('name')['rating_count'].sum().sort_values(ascending=False).index[0]

5th exp - Develop Content( text, emoticons, image, audio, video) based  social media analytics model for business.(local business)
(e.g.Content Based Analysis :Topic , Issue ,Trend, sentiment/opinion analysis, audio, video, image analytics)





In [ ]:
!pip install --quiet --upgrade diffusers transformers accelerate

!pip install torch==2.1.0

!pip install -q xformers==0.0.16rc425

import numpy as np
import pandas as pd
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
from textblob import TextBlob
from diffusers import StableDiffusionPipeline
import torch

nltk.download('all')

nlp = spacy.load('en_core_web_sm')

df=pd.read_csv('/content/Chembur_Social_cleaned_data.csv')

df["Number of Photos"]=df["Number of Photos"].fillna('0 photos')
df["Number of reviews"]=df["Number of reviews"].fillna('0 reviews')
df["Name"]=df["Name"].fillna('Unknown user')

df.dropna(inplace=True)

col = df['Review']

words = col.apply(lambda row: nltk.word_tokenize(row))

word_list = []
for i in words:
  for j in i:
    word_list.append(j)

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
word_list_cleaned = []
word_list_cleaned = [w for w in word_list if not w.lower() in stop_words and w.isalpha()]

for i in range(20):
  doc = nlp(col[i])
  concepts = [(ent.text, ent.label_) for ent in doc.ents]
  for concept, label in concepts:
    print(f"Concept: {concept}, Label: {label}")

reviews = ' '.join(col)
doc1 = nlp(reviews)
trend_terms = [token.text.lower() for token in doc if token.pos_ in ['NOUN', 'PROPN', ] and not token.is_stop]
term_freq = Counter(trend_terms)
print("Top 10 Trending Terms:")
for term, freq in term_freq.most_common(10):
    print(f"{term}: {freq}")

from textblob import TextBlob

topic = 'Service'  # Replace with your specific topic

stance_labels = []

# Check if 'Review' column exists in the DataFrame
if 'Review' in df.columns:
    for i in range(min(50, len(df))):
        comment = df['Review'].iloc[i]
        analysis = TextBlob(comment)

        if topic.lower() in comment.lower():
            if analysis.sentiment.polarity > 0:
                stance_labels.append('Support')
            elif analysis.sentiment.polarity < 0:
                stance_labels.append('Against')
            else:
                stance_labels.append('Neutral')
        else:
            stance_labels.append('Not related')

    # Print the results
    for j in range(len(stance_labels)):
        print(df['Review'].iloc[j], " : ", stance_labels[j])
else:
    print("Error: 'Review' column not found in the DataFrame.")


#model_id = "dreamlike-art/dreamlike-photoreal-2.0"
model_id = "prompthero/openjourney"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

prompt = ['Socials Restaurant and bar with good ambience, great service, it is also a night club with music system and DJ']

images = []
image = pipe(prompt).images[0]
image.save('result1.jpg')



6th exp - Develop Structure based  social media analytics model for any business.
( e.g. Structure Based Models  -community detection,influence analysis)



In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_table("/content/roadNet-PA.txt")
df.head()

df1 = df.head(150)

collab_graph = nx.from_pandas_edgelist(df1, source = 'FromNodeId', target = "ToNodeId")

collab_graph.edges()

G = nx.Graph(collab_graph)

nx.draw(collab_graph, with_labels = True)

nx.degree(collab_graph)

print(nx.degree_centrality(collab_graph), end = " ")

betweenness_centrality = nx.betweenness_centrality(collab_graph)
print("Betweenness Centrality:", betweenness_centrality)
closeness_centrality = nx.closeness_centrality(collab_graph)
print("Closeness Centrality:", closeness_centrality)

pos = nx.spring_layout(G)
betcent = nx.betweenness_centrality(G, normalized = True, endpoints = True)
node_color = [20000.0 * G.degree(v) for v in G]
node_size = [10000.0 * v for v in betcent.values()]
plt.figure(figsize = (10,10))
nx.draw_networkx_edges(G, pos = pos)
plt.show()

communities_gen = nx.algorithms.community.girvan_newman(collab_graph)
top_level = next(communities_gen)
next_level = next(communities_gen)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
pos = nx.spring_layout(collab_graph)
for i, community in enumerate(top_level):
  nx.draw_networkx_nodes(collab_graph, pos, nodelist=community, node_color=f"C{i}", label=f"Community {i}")
nx.draw_networkx_edges(collab_graph, pos, alpha = 0.5)
plt.title("Top Level Communities")
plt.legend()

plt.subplot(1, 2, 2)
pos = nx.spring_layout(collab_graph)
for i, community in enumerate(next_level):
  nx.draw_networkx_nodes(collab_graph, pos, nodelist=community, node_color=f"C{i}", label=f"Community {i}")
nx.draw_networkx_edges(collab_graph, pos, alpha = 0.5)
plt.title("Next Level Communities")
plt.legend()
plt.show()

edge_betweeness = nx.edge_betweenness_centrality(G)

sorted_edges = sorted(edge_betweeness.items(), key = lambda x: x[1], reverse = True)

print("Top bridges:")
for edge, betweeness in sorted_edges[:5]:
  print(edge, "Betweeness centrality:", betweeness)

top_bridges = sorted_edges[:5]

bridges, betweeness = zip(*top_bridges)
bridges = [str(edge) for edge in bridges]
plt.bar(bridges, betweeness)
plt.bar(bridges, betweeness)
plt.xlabel('Bridge/Edge')
plt.ylabel('Edge Betweeness Centrality')
plt.title('Top bridges by Edge Betweeness Centrality')
plt.xticks(rotation=45)
plt.show()

7th exp - Develop a dashboard and reporting tool based on real time social media data. POWERBI



8th exp - DEVELOP ADVERTISING

9th exp

In [ ]:
!pip install vaderSentiment -q

import re
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

#cleaning  reviews
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

def clean_reviews(reviews):
    #remove twitter Return handles (RT @xxx:)
    reviews = np.vectorize(remove_pattern)(reviews, "RT @[\w]*:")

    #remove twitter handles (@xxx)
    reviews = np.vectorize(remove_pattern)(reviews, "@[\w]*")

    #remove URL links (httpxxx)
    reviews = np.vectorize(remove_pattern)(reviews, "https?://[A-Za-z0-9./]*")

    #remove special characters, numbers, punctuations (except for #)
    reviews = np.core.defchararray.replace(reviews, "[^a-zA-Z]", " ")

    return reviews

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
def word_cloud(wd_list):
    stopwords = set(STOPWORDS)
    all_words = ' '.join([text for text in wd_list])
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        width=1600,
        height=800,
        random_state=1,
        colormap='jet',
        max_words=80,
        max_font_size=200).generate(all_words)
    plt.figure(figsize=(12, 10))
    plt.axis('off')
    plt.imshow(wordcloud, interpolation="bilinear");

#Subaru
df1 = pd.read_csv('Scraped_Car_Review_subaru.csv', lineterminator='\n')
df1

df1 = df1.dropna()

#convert array to dataframe
df = pd.DataFrame.from_dict(df1)
df

df['Review'] = clean_reviews(df['Review'])
df

scores = []
# Declare variables for scores
compound_list = []
positive_list = []
negative_list = []
neutral_list = []
for i in range(df['Review'].shape[0]):
#print(analyser.polarity_scores(sentiments_pd['text'][i]))
    if i != 491 and i != 498 and i != 511 and i != 514 and i != 517 and i != 957 and i != 958 and i != 962 and i != 970 and i != 974 and i != 976 and i != 977:
      compound = analyzer.polarity_scores(df['Review'][i])["compound"]
      pos = analyzer.polarity_scores(df['Review'][i])["pos"]
      neu = analyzer.polarity_scores(df['Review'][i])["neu"]
      neg = analyzer.polarity_scores(df['Review'][i])["neg"]
      # print(i)

      scores.append({"Compound": compound,
                        "Positive": pos,
                        "Negative": neg,
                        "Neutral": neu
                    })

sentiments_score = pd.DataFrame.from_dict(scores)
df = df.join(sentiments_score)
df1 = df

df1

word_cloud(df1['Review'])

#Range Rover
df2 = pd.read_csv('/content/Scraped_Car_Review_land-rover.csv',  lineterminator='\n')
df2

df2 = df2.dropna()

#convert array to dataframe
df = pd.DataFrame.from_dict(df2)
df

df['Review'] = clean_reviews(df['Review'])
df

scores = []
# Declare variables for scores
compound_list = []
positive_list = []
negative_list = []
neutral_list = []
for i in range(df['Review'].shape[0]):
#print(analyser.polarity_scores(sentiments_pd['text'][i]))
    # if i != 491 and i != 498 and i != 511 and i != 514 and i != 517 and i != 957 and i != 958 and i != 962 and i != 970 and i != 974 and i != 976 and i != 977:
    compound = analyzer.polarity_scores(df['Review'][i])["compound"]
    pos = analyzer.polarity_scores(df['Review'][i])["pos"]
    neu = analyzer.polarity_scores(df['Review'][i])["neu"]
    neg = analyzer.polarity_scores(df['Review'][i])["neg"]
    # print(i)

    scores.append({"Compound": compound,
                      "Positive": pos,
                      "Negative": neg,
                      "Neutral": neu
                  })

sentiments_score = pd.DataFrame.from_dict(scores)
df = df.join(sentiments_score)
df2 = df

df2

word_cloud(df2['Review'])

for df in [df1, df2]:
    df['Sentiment'] = df['Review'].apply(lambda x: analyzer.polarity_scores(x))
    df['Compound'] = df['Sentiment'].apply(lambda x: x['compound'])
    df['Positive'] = df['Sentiment'].apply(lambda x: x['pos'])
    df['Negative'] = df['Sentiment'].apply(lambda x: x['neg'])
    df['Neutral'] = df['Sentiment'].apply(lambda x: x['neu'])

# Calculate average scores for each DataFrame
df1_avg_scores = df1[['Compound', 'Positive', 'Negative', 'Neutral']].mean()
df2_avg_scores = df2[['Compound', 'Positive', 'Negative', 'Neutral']].mean()

# Compare average scores between the two car brands
print("Average scores for Subaru:")
print(df1_avg_scores)
print("\nAverage scores for Land Rover:")
print(df2_avg_scores)

fig, ax = plt.subplots(figsize=(10, 6))

bar_width = 0.35
index = range(len(df1_avg_scores))

bar1 = ax.bar(index, df1_avg_scores, bar_width, label='Subaru')
bar2 = ax.bar([i + bar_width for i in index], df2_avg_scores, bar_width, label='Land Rover')

ax.set_xlabel('Sentiment')
ax.set_ylabel('Average Score')
ax.set_title('Average Sentiment Scores by Car Brand')
ax.set_xticks([i + bar_width / 2 for i in index])
ax.set_xticklabels(df1_avg_scores.index)
ax.legend()

plt.show()

# **10th exp**

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

df = pd.read_csv('cleanedKFC.csv')

df

df.dropna(subset=['cleaned_review'], inplace=True)
df.drop(df[df['cleaned_review'] == ''].index, inplace=True)

from wordcloud import WordCloud

text = ' '.join(df['cleaned_review'])

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Plot the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Customer Reviews')
plt.show()

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['cleaned_review'])

k = 5  # Choose the number of clusters
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(X)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X.toarray())

df['cluster'] = kmeans.labels_

plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=df['cluster'], palette='viridis', legend='full')
plt.title('Clustering Results')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Cluster')
plt.show()

for cluster_num in range(k):
    cluster_reviews = df[df['cluster'] == cluster_num]['cleaned_review'].sample(5, random_state=42)
    print(f"\nCluster {cluster_num}:")
    for i, review in enumerate(cluster_reviews):
        print(f"Review {i+1}: {review}")

cluster_means = df.groupby('cluster').mean().reset_index()

# Print cluster means
print(cluster_means)

num_cols = 3
num_rows = -(-k // num_cols)  # Ceiling division to ensure enough rows
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 5 * num_rows))

for cluster_id, ax in zip(range(k), axs.flatten()):
    cluster_reviews_text = ' '.join(df[df['cluster'] == cluster_id]['cleaned_review'])
    wordcloud = WordCloud(width=400, height=200, background_color='white').generate(cluster_reviews_text)
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.set_title(f'Cluster {cluster_id}')
    ax.axis('off')

for i in range(k, num_rows * num_cols):
    axs.flatten()[i].axis('off')

plt.tight_layout()
plt.show()


Amazon review

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import math
import warnings
warnings.filterwarnings('ignore') # Hides warning
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore",category=UserWarning)
sns.set_style("whitegrid") # Plotting style
np.random.seed(42) # seeding random number generator

df = pd.read_csv('../input/amazon-product-reviews/amazon.csv')
df.head()

data = df.copy()
data.describe()

data.info()

data["asins"].unique()

asins_unique = len(data["asins"].unique())
print("Number of Unique ASINs: " + str(asins_unique))

data["reviews.numHelpful"].hist(figsize=(20,5))
plt.show()

data["reviews.id"].hist( figsize=(20,5))
plt.show()

plt.figure(figsize=(20,5))
ax1=sns.countplot(x="reviews.rating", data=data)
for p in ax1.patches:
    ax1.annotate(str(p.get_height()), (p.get_x() * 1.01 , p.get_height() * 1.01))
plt.show()


from sklearn.model_selection import StratifiedShuffleSplit
print("Before : {}".format(len(data)))
dataAfter = data.dropna(subset=["reviews.rating"])
# Removes all NAN in reviews.rating
print("After  : {}".format(len(dataAfter)))
dataAfter["reviews.rating"] = dataAfter["reviews.rating"].astype(int)

split = StratifiedShuffleSplit(n_splits=10, test_size=0.2)
for train_index, test_index in split.split(dataAfter,
                                           dataAfter["reviews.rating"]):
    strat_train = dataAfter.reindex(train_index)
    strat_test = dataAfter.reindex(test_index)


print(len(strat_train))
print(len(strat_test))
round((strat_test["reviews.rating"].value_counts()* 100/len(strat_test)),2)

reviews = strat_train.copy()
reviews.head()

len(reviews["name"].unique())

len(reviews["asins"].unique())

reviews.info()

reviews.groupby("asins")["name"].unique()

different_names = reviews[reviews["asins"] ==
                          "B00L9EPT8O,B01E6AO69U"]["name"].unique()
for name in different_names:
    print(name)


reviews[reviews["asins"] == "B00L9EPT8O,B01E6AO69U"]["name"].value_counts()

fig = plt.figure(figsize=(16,10))
ax1 = plt.subplot(211)
ax2 = plt.subplot(212, sharex = ax1)
reviews["asins"].value_counts().plot(kind="bar", ax=ax1, title="ASIN Frequency")
np.log10(reviews["asins"].value_counts()).plot(kind="bar", ax=ax2,
                                               title="ASIN Frequency (Log10 Adjusted)")
for p in ax1.patches:
    ax1.annotate(str(p.get_height()), (p.get_x() * 1.01 , p.get_height() * 1.01))
for p in ax2.patches:
    ax2.annotate(str(round((p.get_height()),2)), (p.get_x() * 1.01 , p.get_height() * 1.01))
plt.show()

reviews["reviews.rating"].mean()

asins_count_ix = reviews["asins"].value_counts().index
fig = plt.figure(figsize=(16,5))
ax=reviews["asins"].value_counts().plot(kind="bar", title="ASIN Frequency")
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.01 , p.get_height() * 1.01))
plt.xticks(rotation=90)
plt.show()

fig = plt.figure(figsize=(16,5))
sns.pointplot(x="asins", y="reviews.rating", order=asins_count_ix, data=reviews)
plt.xticks(rotation=90)
plt.show()

def sentiments(rating):
    if (rating == 5) or (rating == 4):
        return "Positive"
    elif rating == 3:
        return "Neutral"
    elif (rating == 2) or (rating == 1):
        return "Negative"
# Add sentiments to the data
strat_train["Sentiment"] = strat_train["reviews.rating"].apply(sentiments)
strat_test["Sentiment"] = strat_test["reviews.rating"].apply(sentiments)
print(strat_train["Sentiment"][:15])

round((strat_train["Sentiment"].value_counts()*100/len(strat_train)),2)

fig = plt.figure(figsize=(16,5))
ax=strat_train["Sentiment"].value_counts().plot(kind="bar", title="Train Data Sentimental Data")
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.01 , p.get_height() * 1.01))
plt.show()

round((strat_test["Sentiment"].value_counts()*100/len(strat_test)),2)

fig = plt.figure(figsize=(16,5))
ax=strat_test["Sentiment"].value_counts().plot(kind="bar", title="Test Data Sentimental Data")
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.01 , p.get_height() * 1.01))
plt.show()



Expt 10

In [ ]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

df = pd.read_csv('/content/The Habitat review cleaned data (1).csv')  # Update with your file path

df['Review'] = df['Review'].fillna('')

comments = df['Review'].tolist()

def analyze_sentiment(text):
    if pd.isna(text):  # Handle missing values
        return 'neutral'
    elif isinstance(text, str):  # Check if text is already a string
        blob = TextBlob(text)
        sentiment_score = blob.sentiment.polarity
        return 'positive' if sentiment_score > 0 else 'negative' if sentiment_score < 0 else 'neutral'
    else:
        return 'neutral'

df['sentiment'] = df['Review'].apply(analyze_sentiment)

X_train, X_test, y_train, y_test = train_test_split(comments, df['sentiment'], test_size=0.2, random_state=42)

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

vectorizer = TfidfVectorizer(max_features=1000)  # Adjust max_features as needed
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

xgb = XGBClassifier()
xgb.fit(X_train_vec, y_train_encoded)

y_pred_encoded = xgb.predict(X_test_vec)

y_pred = label_encoder.inverse_transform(y_pred_encoded)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Classification Report:")
print(classification_rep)

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

kmeans = KMeans(n_clusters=3)
kmeans.fit(X_train_vec)
clusters = kmeans.predict(X_test_vec)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_test_vec.toarray())

cluster_samples = [[] for _ in range(3)]

for sample, cluster_label in zip(X_test, clusters):
    cluster_samples[cluster_label].append(sample)

for i, samples in enumerate(cluster_samples):
    print(f"Cluster {i} Samples:")
    for j, sample in enumerate(samples[:5]):
        print(f"Sample {j+1}: {sample}")
    print()

plt.figure(figsize=(10, 8))
for i in range(3):
    plt.scatter(X_pca[clusters == i, 0], X_pca[clusters == i, 1], label=f'Cluster {i}')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('K-Means Clustering of Reviews (PCA)')
plt.legend()
plt.show()